In [ ]:
import os
import sys
import urllib.request
import platform
import calendar
import requests
import datetime
import time
import json
import re
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from multiprocessing import Process
import csv

In [ ]:
client_id = "ID"
client_secret = "PW"

title1 = []
pDate1 = []
description1 = []
org_link1 =[]
link1 = []
dt_now = datetime.datetime.now().strftime('%Y-%m-%d')

#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s]Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e :
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)     #[CODE 1]

    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostData(post, jsonResult, cnt):
    cleanr = re.compile('<.*?>')

    title = post['title']
    titlec = re.sub(cleanr, '', title)
    description = post['description']
    descriptionc = re.sub(cleanr, '', description)
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    title1.append(titlec)
    description1.append(descriptionc)
    org_link1.append(org_link)
    link1.append(link)
    pDate1.append(pDate)

    jsonResult.append({'cnt':cnt, 'title':title, 'description':description,
                       'org_link':org_link, 'link':org_link, 'pDate':pDate})
    return

#[CODE 0]
def main():
    node = 'news'  #크롤링한 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']

    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt+= 1
            getPostData(post, jsonResult, cnt)   #[CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)   #[CODE 2]

    print('전체 검색 : %d 건' %total)

    #with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
    #    jsonFile = json.dumps(jsonResult, indent=4, sort_keys = True, ensure_ascii = False)

    #    outfile.write(jsonFile)

    df=pd.DataFrame([title1, pDate1, description1, org_link1, link1 ]).T
    df.columns=['제목','날짜','내용','원래주소','네이버뉴스주소']

    print("가져온 데이터 : %d 건" %(cnt))
    #print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()


In [ ]:
post_urls = link1

In [ ]:
# 데이터 클렌징
class ArticleParser(object):
    special_symbol = re.compile('[\{\}\[\]\/?,;:|\)*~`!^\-_+<>@\#$&▲▶◆◀■【】\\\=\(\'\"]')
    content_pattern = re.compile('본문 내용 | TV플레이어 | 동영상 뉴스 | flash 오류를 우회하기 위한 함수 추가function  flash removeCallback|tt | 앵커 멘트 | xa0')

    @classmethod
    def clear_content(cls, text):
        # 기사 본문에서 필요없는 특수문자 및 본문 양식 등을 다 지움
        newline_symbol_removed_text = text.replace('\\n', '').replace('\\t', '').replace('\\r', '') #줄바꿈, 탭, 커서를 앞의행으로 이동 제거
        special_symbol_removed_content = re.sub(cls.special_symbol, ' ', newline_symbol_removed_text) #특수문자 제거
        end_phrase_removed_content = re.sub(cls.content_pattern, '', special_symbol_removed_content) #일정 패턴 제거
        blank_removed_content = re.sub(' +', ' ', end_phrase_removed_content).lstrip()  # 공백 에러 삭제
        reversed_content = ''.join(reversed(blank_removed_content))  # 기사 내용을 reverse 한다.
        content = ''
        for i in range(0, len(blank_removed_content)):
            # reverse 된 기사 내용중, ".다"로 끝나는 경우 기사 내용이 끝난 것이기 때문에 기사 내용이 끝난 후의 광고, 기자 등의 정보는 다 지움
            if reversed_content[i:i + 2] == '.다':
                content = ''.join(reversed(reversed_content[i:]))
                break
        return content

    @classmethod
    def clear_headline(cls, text):
        # 기사 제목에서 필요없는 특수문자들을 지움
        newline_symbol_removed_text = text.replace('\\n', '').replace('\\t', '').replace('\\r', '')
        special_symbol_removed_headline = re.sub(cls.special_symbol, '', newline_symbol_removed_text)
        return special_symbol_removed_headline

In [ ]:
# 오류링크 제거
search = "sid=106"
for word in post_urls:
    if search in word:
        post_urls.remove(word)

search = "0004316027?sid=105"
for word in post_urls:
    if search in word:
        post_urls.remove(word)


In [ ]:
file_list = []
for content_url in post_urls:
  # 신문사 구분 link 뽑아내기
  title = content_url.split('/')[2]
  print(content_url)
  # 크롤링 대기 시간
  sleep(0.01)

#nnews naver
  if title == 'n.news.naver.com':
    request_content = requests.get(content_url)
    document_content = BeautifulSoup(request_content.content, 'html.parser')

    # 기사 제목 가져옴
    tag_headline = document_content.find_all('h2', {'id': 'title_area'})
    # 뉴스 기사 제목 초기화
    text_headline = ''
    text_headline = text_headline + ArticleParser.clear_headline(str(tag_headline[0].find_all(text=True)))
    print(text_headline)
    # 공백일 경우 기사 제외 처리
    if not text_headline:
        continue

    # 기사 본문 가져옴
    tag_content = document_content.find_all('div', {'id': 'contents'})
    # 뉴스 기사 본문 초기화
    text_sentence = ''
    text_sentence = text_sentence + ArticleParser.clear_content(str(tag_content[0].find_all(text=True)))
    print(text_sentence)
    # 공백일 경우 기사 제외 처리
    if not text_sentence:
        continue

    # 기사 언론사 가져옴
    tag_company = document_content.find_all('meta', {'property': 'og:article:author'})
    # 언론사 초기화
    text_company = ''
    text_company = text_company + str(tag_company[0].get('content'))
    print(text_company)
    # 공백일 경우 기사 제외 처리
    if not text_company:
        continue

    # data append to list
    data = [text_company, text_headline, text_sentence, content_url]
    file_list .append(data)

#인포멕스
  elif title == 'news.einfomax.co.kr':
    request_content = requests.get(content_url)
    document_content = BeautifulSoup(request_content.content, 'html.parser')

    # 기사 제목 가져옴
    tag_headline = document_content.find_all('h3', {'class': 'heading'})
    # 뉴스 기사 제목 초기화
    text_headline = ''
    text_headline = text_headline + ArticleParser.clear_headline(str(tag_headline[0].find_all(text=True)))
    print(text_headline)
    # 공백일 경우 기사 제외 처리
    if not text_headline:
        continue

    # 기사 본문 가져옴
    tag_content = document_content.find_all('article', {'id': 'article-view-content-div'})
    # 뉴스 기사 본문 초기화
    text_sentence = ''
    text_sentence = text_sentence + ArticleParser.clear_content(str(tag_content[0].find_all(text=True)))
    print(text_sentence)
    # 공백일 경우 기사 제외 처리
    if not text_sentence:
        continue

    # 기사 언론사 가져옴
    tag_company = document_content.find_all('meta', {'name': 'apple-mobile-web-app-title'})
    # 언론사 초기화
    text_company = ''
    text_company = text_company + str(tag_company[0].get('content'))
    print(text_company)
    # 공백일 경우 기사 제외 처리
    if not text_company:
        continue

    # data append to list
    data = [text_company, text_headline, text_sentence, content_url]
    file_list .append(data)

  else:
    pass

In [ ]:
from pandas import DataFrame

#list to dataframe
df = DataFrame (file_list,columns=['text_company','text_headline','text_sentence','content_url'])
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
df.to_csv('/content/drive/MyDrive/Colab Notebooks/amazon_craw.csv')
